In [1]:
%pylab inline

import pandas as pd
import numpy as np
import google_trans_new
import nltk
import re

#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")
nltk.download('wordnet')

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from gensim.models import Phrases
from gensim.corpora import Dictionary

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from google_trans_new import google_translator
translator = google_translator()

from yellowbrick.cluster.elbow import kelbow_visualizer

from scipy import sparse as sp

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/opt/conda/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_excel('/home/jovyan/work/data/b2b_csat/monthly_csat.xlsx')

for i,text_lines in enumerate(df['NPS Reason']):
    df.loc[i,'nps_reason_eng'] = translator.translate(text_lines, lang_src='ar', lang_tgt='en')
   # print(df.loc[i,'nps_reason_eng'])

### Basic EDA


In [3]:
df.head(3)

,Company/Institution/Government Name,Name,Mobile Number,Month,Segmnet,NPS,NPS Category,NPS Reason,Overall CSAT,Product creation CSAT,...,Fixed,Mobility,Data,Managed Services,Computing,Cybersecurity,Big Data,IoT,PTT,nps_reason_eng
0,Royal Protocol,ياسر العتيبي,500154342,Jan,GOV,10,Promotor,شركة وطنيه في المقام الأول ، وخدمت مميزة على م...,1.000000,1.0,...,1,1,1,1,0,0,0,0,0,A national company in the first place and serv...
1,Saudi Credit and Savings Bank,فهد القرني,500525086,Jan,GOV,9,Promotor,تعامل راقي ، ودعم فني مميز ، وجودة خدمات مميزة,0.953333,NaN,...,1,0,1,1,0,0,0,0,0,"Ultrasonic treatment, special technical suppor..."
2,Technical and Vocational Training Corporation,Mohamed Almana,504101753,Jan,GOV,8,Passive,جودة خدمات لاتجدها في الشركات المنافسة ، ولكن ...,0.950000,1.0,...,1,0,1,1,0,1,0,0,0,Quality of services do not find in competitive...


In [4]:
df.rename({'NPS Category':'nps_cat'},axis=1,inplace=True)
df_dist = df.groupby(['Month','nps_cat']).agg({'nps_cat':'count'})
df_dist.rename({'nps_cat':'nps_count'},axis=1,inplace=True)
df_dist.reset_index(inplace=True)
df_dist

,Month,nps_cat,nps_count
0,Apr,Detractor,5
1,Apr,Passive,13
2,Apr,Promotor,42
3,Feb,Detractor,7
4,Feb,Passive,17
5,Feb,Promotor,38
6,Jan,Detractor,4
7,Jan,Passive,8
8,Jan,Promotor,48
9,Mar,Detractor,11


### Text Analysis 

In [5]:
p_df = df['nps_reason_eng']
p_df

0      A national company in the first place and serv...
1      Ultrasonic treatment, special technical suppor...
2      Quality of services do not find in competitive...
3      Mimiz Communications Company that all services...
4      Wonderful services, speed in the Internet desp...
                             ...                        
237                There are no comments from customers 
238                There are no comments from customers 
239    Please modify the mandate We are currently ren...
240    The application of portal has two explanations...
241                There are no comments from customers 
Name: nps_reason_eng, Length: 242, dtype: object

In [6]:
docs = array(p_df)
docs

array(['A national company in the first place and served a distinctive safety and quality ',
       'Ultrasonic treatment, special technical support, and quality of special services ',
       'Quality of services do not find in competitive companies, but prices are slightly high in electronic circuit packages ',
       'Mimiz Communications Company that all services and products are permanently available ',
       'Wonderful services, speed in the Internet despite having some slow peak times, but the company is generally special in its services, products and deals ',
       "Coverage of the network is bad in some border places for Saudi Arabia's outlets with other countries, but as great and distinctive products. ",
       'Excellent sales team, quality and excellent services ',
       'Excellent dealing from everyone ',
       'Quality of service, attention and excellent dealing from everyone ',
       'Excellent services ', 'There are no comments from customers ',
       'Service of 

### Pre-process and vectorize the documents

In [7]:
def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [8]:
docs = docs_preprocessor(docs)

### Compute bigrams/trigrams: 

In [9]:
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

###  Remove rare and common tokens

In [10]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 571
Number of unique words after removing rare and common words: 28


### Vectorize data 

In [11]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 28
Number of documents: 242


### Train LDA model 

In [34]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 4
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


CPU times: user 1.97 s, sys: 6.31 ms, total: 1.98 s
Wall time: 1.97 s


### Number of Topics 

In [35]:
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.235595  0.179035       1        1  33.226343
3      0.059052 -0.217136       2        1  23.080001
0      0.017587 -0.052941       3        1  22.796461
2     -0.312233  0.091042       4        1  20.897195, topic_info=               Term       Freq      Total Category  logprob  loglift
14  account_manager  61.000000  61.000000  Default  28.0000  28.0000
11        excellent  45.000000  45.000000  Default  27.0000  27.0000
13          account  35.000000  35.000000  Default  26.0000  26.0000
22           number  22.000000  22.000000  Default  25.0000  25.0000
15             been  30.000000  30.000000  Default  24.0000  24.0000
..              ...        ...        ...      ...      ...      ...
13          account   0.343656  35.555952   Topic4  -5.8429  -3.0737
6              that   0.246868  26.056753   Topic4  -6.1737  -3.0936
21           client   0.252971  31.769897   Topic4  -6.1493  -3.2674
15             been   0.239043  30.069302   Topic4  -6.2059  -3.2691
14  account_manager   0.284836  61.528649   Topic4  -6.0306  -3.8098

[140 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
13        1  0.956239          account
13        3  0.028125          account
14        1  0.991408  account_manager
15        2  0.332565             been
15        3  0.665130             been
16        3  0.905014         business
3         2  0.191694          circuit
3         3  0.095847          circuit
3         4  0.670928          circuit
21        1  0.346240           client
21        2  0.440669           client
21        3  0.220334           client
0         2  0.548250          company
0         4  0.426417          company
8         4  0.909718         coverage
1         2  0.746415      distinctive
1         4  0.279906      distinctive
24        1  0.169733            email
24        2  0.792088            email
11        3  0.283798        excellent
11        4  0.698579        excellent
17        1  0.961388          manager
17        3  0.040058          manager
9         2  0.075553          network
9         4  0.906639          network
22        3  0.951670           number
23        2  0.246461          problem
23        3  0.739382          problem
4         1  0.088475          product
4         3  0.088475          product
4         4  0.796278          product
2         1  0.075683          quality
2         4  0.832508          quality
18        1  0.678068         reported
18        3  0.301363         reported
27        1  0.702825    reported_that
27        3  0.263559    reported_that
19        2  0.844078          request
19        3  0.120583          request
20        1  0.098271           sector
20        3  0.786165           sector
25        2  0.954769             sent
5         1  0.096499  service_product
5         3  0.096499  service_product
5         4  0.771994  service_product
6         1  0.422155             that
6         2  0.422155             that
6         3  0.153511             that
7         4  0.897960             time
12        1  0.418533             very
12        2  0.279022             very
12        3  0.279022             very
26        1  0.056388             will
26        2  0.958603             will
10        1  0.406313             with
10        2  0.135438             with
10        3  0.379225             with
10        4  0.054175             with, R=28, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

In [14]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(model.print_topics())
doc_lda = model[corpus]

[(0,
  '0.201*"account_manager" + 0.141*"with" + 0.092*"account" + 0.076*"manager" '
  '+ 0.074*"that" + 0.069*"been" + 0.067*"reported" + 0.061*"client" + '
  '0.050*"reported_that" + 0.041*"request"'),
 (1,
  '0.393*"been" + 0.271*"problem" + 0.175*"client" + 0.047*"time" + '
  '0.037*"email" + 0.004*"number" + 0.003*"will" + 0.003*"sent" + '
  '0.003*"account" + 0.003*"circuit"'),
 (2,
  '0.184*"product" + 0.168*"service_product" + 0.094*"that" + 0.090*"company" '
  '+ 0.086*"reported" + 0.086*"reported_that" + 0.081*"client" + '
  '0.071*"quality" + 0.066*"circuit" + 0.043*"with"'),
 (3,
  '0.242*"distinctive" + 0.235*"company" + 0.106*"with" + 0.099*"very" + '
  '0.075*"that" + 0.060*"client" + 0.058*"manager" + 0.029*"network" + '
  '0.026*"will" + 0.026*"been"'),
 (4,
  '0.269*"excellent" + 0.207*"coverage" + 0.190*"network" + 0.160*"business" + '
  '0.137*"sector" + 0.002*"very" + 0.002*"with" + 0.002*"reported" + '
  '0.002*"reported_that" + 0.002*"that"'),
 (5,
  '0.198*"sent

In [15]:
p_df = pd.DataFrame(p_df)
p_df

,nps_reason_eng
0,A national company in the first place and serv...
1,"Ultrasonic treatment, special technical suppor..."
2,Quality of services do not find in competitive...
3,Mimiz Communications Company that all services...
4,"Wonderful services, speed in the Internet desp..."
...,...
237,There are no comments from customers
238,There are no comments from customers
239,Please modify the mandate We are currently ren...
240,The application of portal has two explanations...


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

p_df['tokenz'] = docs

docs1 = p_df['tokenz'].apply(lambda l: l[:int0(len(l)/2)])
docs2 = p_df['tokenz'].apply(lambda l: l[int0(len(l)/2):])



In [17]:
p_df

,nps_reason_eng,tokenz
0,A national company in the first place and serv...,"[national, company, first, place, served, dist..."
1,"Ultrasonic treatment, special technical suppor...","[ultrasonic, treatment, special, technical, su..."
2,Quality of services do not find in competitive...,"[quality, service, find, competitive, company,..."
3,Mimiz Communications Company that all services...,"[mimiz, communication, company, that, service,..."
4,"Wonderful services, speed in the Internet desp...","[wonderful, service, speed, internet, despite,..."
...,...,...
237,There are no comments from customers,"[there, comment, from, customer, there_comment..."
238,There are no comments from customers,"[there, comment, from, customer, there_comment..."
239,Please modify the mandate We are currently ren...,"[please, modify, mandate, currently, renewed, ..."
240,The application of portal has two explanations...,"[application, portal, explanation, every, mont..."


In [18]:
corpus1 = [dictionary.doc2bow(doc) for doc in docs1]
corpus2 = [dictionary.doc2bow(doc) for doc in docs2]

# Using the corpus LDA model tranformation
lda_corpus1 = model[corpus1]
lda_corpus2 = model[corpus2]

In [19]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=False):
    
    '''
    LDA transformation, for each doc only returns topics with non-zero weight
    This function makes a matrix transformation of docs in the topic space.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]

    return array(top_dist), keys



In [20]:
top_dist1, _ = get_doc_topic_dist(model, lda_corpus1)
top_dist2, _ = get_doc_topic_dist(model, lda_corpus2)

print("Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):")
print(mean([cosine_similarity(c1.reshape(1, -1), c2.reshape(1, -1))[0][0] for c1,c2 in zip(top_dist1, top_dist2)]))

random_pairs = np.random.randint(0, len(p_df['nps_reason_eng']), size=(400, 2))

print("Inter similarity: cosine similarity between random parts (lower is better):")
print(np.mean([cosine_similarity(top_dist1[i[0]].reshape(1, -1), top_dist2[i[1]].reshape(1, -1)) for i in random_pairs]))

Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):
0.8892632
Inter similarity: cosine similarity between random parts (lower is better):
0.788369


In [21]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [22]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                 frequency

Topic 0 |---------------------

account_manager      0.201
with                 0.141
account              0.092
manager              0.076
that                 0.074
been                 0.069
reported             0.067
client               0.061
reported_that        0.050
request              0.041
Topic 1 |---------------------

been                 0.393
problem              0.271
client               0.175
time                 0.047
email                0.037
number               0.004
will                 0.003
sent                 0.003
account              0.003
circuit              0.003
Topic 2 |---------------------

product              0.184
service_product      0.168
that                 0.094
company              0.090
reported             0.086
reported_that        0.086
client               0.081
quality              0.071
circuit              0.066
with                 0.043
Topic 3 |---------------------

distinctive          0.242
com

In [23]:
top_labels = {0: 'one', 1:'two', 2:'three', 3:'four'}

In [24]:
import re
import nltk

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

def paper_to_wordlist( paper, remove_stopwords=True ):
    '''
        Function converts text to a sequence of words,
        Returns a list of words.
    '''
    lemmatizer = WordNetLemmatizer()
    # 1. Remove non-letters
    paper_text = re.sub("[^a-zA-Z]"," ", paper)
    # 2. Convert words to lower case and split them
    words = paper_text.lower().split()
    # 3. Remove stop words
    words = [w for w in words if not w in stops]
    # 4. Remove short words
    words = [t for t in words if len(t) > 2]
    # 5. lemmatizing
    words = [nltk.stem.WordNetLemmatizer().lemmatize(t) for t in words]

    return(words)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvectorizer = TfidfVectorizer(input='content', analyzer = 'word', lowercase=True, stop_words='english',\
                                  tokenizer=paper_to_wordlist, ngram_range=(1, 3), min_df=40, max_df=0.20,\
                                  norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

dtm = tvectorizer.fit_transform(p_df['nps_reason_eng']).toarray()

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [26]:
top_dist =[]
for d in corpus:
    tmp = {i:0 for i in range(num_topics)}
    tmp.update(dict(model[d]))
    vals = list(OrderedDict(tmp).values())
    top_dist += [array(vals)]

In [27]:
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)
features = tvectorizer.get_feature_names()

In [28]:
top_ws = []
for n in range(len(dtm)):
    inds = int0(argsort(dtm[n])[::-1][:4])
    tmp = [features[i] for i in inds]
    
    top_ws += [' '.join(tmp)]
    
p_df['Text_Rep'] = pd.DataFrame(top_ws)
p_df['clusters'] = pd.DataFrame(lda_keys)
p_df['clusters'].fillna(10, inplace=True)

#cluster_colors = {'Promotor': 'blue', 'Detractor': 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

cluster_colors = {'Promotor': 'green', 'Detractor': 'red', 'Passive': 'yellow'}

p_df['colors'] = df['nps_cat'].apply(lambda l: cluster_colors[l])

In [29]:
p_df.groupby(['clusters','colors']).agg({'colors':'count'})

colors
clusters colors        
0        green        3
         red          5
1        green        1
         red          4
         yellow       2
2        green        8
         yellow       2
3        green        5
         red          1
         yellow       3
4        green      110
         red          7
         yellow      31
5        green        4
         red          2
         yellow       3
6        green        2
         red          1
         yellow       2
7        green        3
         red          1
         yellow       4
8        green       10
         red          6
         yellow       1
9        green       19
         yellow       2

In [30]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

In [31]:
p_df['X_tsne'] =X_tsne[:, 0]
p_df['Y_tsne'] =X_tsne[:, 1]

In [32]:
from bokeh.plotting import figure, show, output_notebook, save#, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [37]:
source = ColumnDataSource(dict(
    x=p_df['X_tsne'],
    y=p_df['Y_tsne'],
    color=p_df['colors'],
    label=p_df['clusters'].apply(lambda l: top_labels[9]),
#     msize= p_df['marker_size'],
    topic_key= p_df['clusters'],
 #   title= p_df[u'Title'],
    content = p_df['Text_Rep']
))

KeyError: 9

In [ ]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=600
                   #  ,title=title
                  , tools="pan,wheel_zoom,box_zoom,reset,hover,save",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source,
                 color='color', alpha=0.8, size=10)#'msize', )

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title, KeyWords: @content - Topic: @topic_key "}
plot_lda.legend.location = "top_left"

show(plot_lda)

#save the plot
# save(plot_lda, '{}.html'.format(title))